In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install trulens_eval
!pip install llama-index.core
!pip install llama_index.llms.openai
!pip install llama-index
!pip install openai
!pip install utils
!pip install llama-index-embeddings-huggingface

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.5/185.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.5/520.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.3/706.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 M

In [ ]:
from google.colab import userdata
import utils
import os
import openai
openai.api_key=userdata.get('OPENAI_API_KEY');

In [ ]:
#!pip install -U llama-index-readers-file

In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["/content/acad_55.pdf"]
).load_data()

In [ ]:
my_questions = []
with open('iith_questions_50.txt', 'r') as file:
    for line in file:
        item = line.strip()
        my_questions.append(item)

In [ ]:
from trulens_eval import TruLlama
from trulens_eval import Feedback, Select
from trulens_eval.feedback import GroundTruthAgreement

import numpy as np
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [ ]:
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

In [ ]:
import nest_asyncio
from trulens_eval import OpenAI as fOpenAI

nest_asyncio.apply()

provider = fOpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [7]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
context_selection = TruLlama.select_source_nodes().node.text

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

f_qs_relevance = (
    Feedback(provider.qs_relevance, name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
  .on(context_selection)
  .on_output()
)
#Select.RecordCalls.retrieve.rets.collect

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [10]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [8]:
# Sentence window node parser w/ default settings
node_parser=SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",)

In [ ]:
from llama_index.core import ServiceContext
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model="local:BAAI/bge-small-en-v1.5"
Settings.node_parser = node_parser

In [12]:
from llama_index.core import Document
document = Document(text="\n\n".
          join([doc.text for doc in documents]))

In [ ]:
from llama_index.core import VectorStoreIndex
sentence_index = VectorStoreIndex.from_documents(
    [document], llm=Settings.llm,
    embed_model=Settings.embed_model,)

In [15]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank

def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6, rerank_top_n=2,):
    postproc = MetadataReplacementPostProcessor(
        target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n,
        model="local:BAAI/bge-reranker-base"
    )
    s_window_engine=sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k,
        node_postprocessors=[postproc, rerank]
    )
    return s_window_engine

In [ ]:
sentence_window_engine=get_sentence_window_query_engine(sentence_index)

OSError: Incorrect path_or_model_id: 'local:BAAI/bge-reranker-base'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
#tru_recorder = TruLlama(sentence_window_engine,app_id="Sentence_Win",feedbacks=[f_qa_relevance,f_qs_relevance,f_groundedness])
tru_recorder = TruLlama(sentence_window_engine,app_id="Sentence_Win",feedbacks=[f_groundedness])
print(tru_recorder)

In [ ]:
index = VectorStoreIndex.from_documents([document], embed_model=Settings.embed_model, llm=Settings.llm)
query_engine = index.as_query_engine()

In [ ]:
print(tru)
tru.add_feedback=Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness").on(context_selection)  .on_output()

with tru_recorder as recording:
#    for question in my_questions:
#        response = query_engine.query(question)
  response = query_engine.query("Number of credits required to complete the BTech ")
  print(response)
  print(recording)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
The number of credits required to complete the BTech degree is between 125 and 130 credits.


In [ ]:
from trulens_eval.utils.display import get_feedback_result
from trulens_eval import Feedback
from trulens_eval import TruCustomApp
tru_app = TruCustomApp(llm_app, app_id = 'LLM App v1', feedbacks = [f_groundtruth])
records, feedback = tru.get_records_and_feedback(app_ids=[])
print(records.head().columns)
print(feedback)
#get_feedback_result(records.record_id[0],"Context Relevance")
tru.get_leaderboard(app_ids=[])

In [ ]:
import pandas as pd

# creating the DataFrame
#rag_feedback = pd.DataFrame({'QID','Input','Output','Answer_relevance','Context_Relevance','Groundedness'})
rag_feedback = pd.DataFrame(records)

# saving the excel
rag_feedback.to_excel (r'rag_sentence_feedback.xlsx', index = None, header=True)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.19.116.94



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>